## 영어 토큰화

In [ ]:
en_text = "A Dog Run back conrner near spare bedrooms"

#### 1. spaCy

In [ ]:
import spacy
spacy_en = spacy.load('en_core_web_sm')

In [ ]:
def tokenize(en_text):
    return [tok.text for tok in spacy_en.tokenizer(en_text)]

In [ ]:
print(tokenize(en_text))

#### 2. NLTK

In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
from nltk.tokenize import word_tokenize
print(word_tokenize(en_text))

#### 3. 띄어쓰기 토큰화

In [ ]:
print(en_text.split(' '))

## 한국어 토큰화

#### 1. 띄어쓰기 토큰화

In [ ]:
kor_text = "사과의 놀라운 효능이라는 글을 봤어. 그래서 오늘 사과를 먹으려고 했는데 사과가 썩어서 슈퍼에 가서 사과랑 오렌지랑 사왔어."

In [ ]:
# '사과' 단어가 4번 등장했는데 조사로 인해 다 다른 단어로 구분

print(kor_text.split())

#### 2. 형태소 토큰화

In [ ]:
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)


In [ ]:
from konlpy.tag import Mecab
tokenizer = Mecab()
print(tokenizer.morphs(kor_text))

## 문자 토큰화

In [ ]:
print(list(en_text))

## 단여 집합(Vocabulary) 생성

In [ ]:
import urllib.request
import pandas as pd
from konlpy.tag import Mecab
from nltk import FreqDist
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

In [ ]:
data = pd.read_table('ratings.txt')
data[:10]

In [ ]:
print('전체 샘플 수: {}'.format(len(data)))

In [ ]:
sample_data = data[:100]

In [ ]:
sample_data

In [ ]:
# 불용어 정의
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
tokenizer = Mecab()
tokenized = []
for sentence in sample_data['document']:
    temp = tokenizer.morphs(sentence)
    temp = [word for word in temp if not word in stopwords]

    tokenized.append(temp)

In [ ]:
print(tokenized[:10])

In [ ]:
# np.hstack(a): a배열을 일렬로 붙임
# np.hstack(a,b): a와 b의 행(row)크기가 같을 때 옆으로 이어붙임
vocab = FreqDist(np.hstack(tokenized))

In [ ]:
print('단어 집합의 크기: {}'.format(len(vocab)))

In [ ]:
vocab['재미']

In [ ]:
# 빈도 높은 단어 순으로 500개로 압축
vocab_size = 500
vocab = vocab.most_common(vocab_size)
print('단어 집합의 크기: {}'.format(len(vocab)))

## 각 단어에 고유한 정수 부여(인코딩)

- pad(padding): 문장의 길이를 맞추기 위해 넣는 가짜 토큰
- unk(Unknown): 사전에 없는 단어를 대신하는 가짜 토큰

In [ ]:
word_to_index = {word[0] : index + 2 for index, word in enumerate(vocab)}
word_to_index['pad'] = 1
word_to_index['unk'] = 0

In [ ]:
encoded = []
for line in tokenized:
    temp = []
    for w in line:
        try:
            temp.append(word_to_index[w])
        except KeyError:
            temp.append(word_to_index['unk'])
    encoded.append(temp)

In [ ]:
print(encoded[:10])

In [ ]:
max_len = max(len(l) for l in encoded)
print('리뷰의 최대 길이 : %d' % max_len)
print('리뷰의 최소 길이 : %d' % min(len(l) for l in encoded))
print('리뷰의 평균 길이 : %f' % (sum(map(len, encoded))/len(encoded)))
plt.hist([len(s) for s in encoded], bins=50)
plt.xlabel('length of sample')
plt.ylabel('number of sample')
plt.show()

In [ ]:
for line in encoded:
    if len(line) < max_len:
        line += [word_to_index['pad']] * (max_len - len(line))